<a href="https://colab.research.google.com/github/thiyaneshford/LLM/blob/main/BQ_Langchain_with_Embeddings_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Creation of the vector database with FewShot Examples

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user --quiet
!pip install chromadb==0.3.26 pydantic==1.10.8 typing-inspect==0.8.0 typing_extensions==4.5.0 pandas datasets google-api-python-client pypdf faiss-cpu transformers config --upgrade --user --quiet

In [ ]:
# Below libraries are required to build a SQL engine for BigQuery
!pip install --user SQLAlchemy==1.4.48 --quiet
!pip install --user sqlalchemy-bigquery --quiet
# Install pandas
! pip install --user --quiet pandas

# Install HuggingFace Datasets
! pip install --user --quiet datasets

# Install Python client for Google Search API
! pip install --user --quiet google-api-python-client

# PDF loader
! pip install --user --quiet pypdf

# Install similarity search library
# TO-DO: Replace with vertex AI Matching Engine
! pip install --user --quiet faiss-cpu

In [ ]:
!pip install -U langchain langchain_experimental

In [1]:
PROJECT_ID = "iron-decorator-297513" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
from google.cloud import aiplatform
from langchain.llms import VertexAI
import vertexai
aiplatform.init(project=PROJECT_ID, location=LOCATION)
vertexai.init(project=PROJECT_ID, location=LOCATION)
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [2]:
project_id = "iron-decorator-297513" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
dataset_id = 'tg_Dataste' # @param {type:"string"}
table_name = 'LLMsales' # @param {type:"string"}
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import pandas as pd

In [3]:
table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(f"bigquery://{project_id}/{dataset_id}")

In [30]:
#importing the Libraries
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.agents.agent_toolkits import create_retriever_tool
import pandas as pd
from langchain.schema import BasePromptTemplate
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma

In [15]:
template = """ Given an input question, first create a syntactically correct bigquery  to run, then look at the results of the query and return the answer.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL
Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

when ever the question is about Vehicle_ID first do the join with the Vehicle_ID table using Vehicle number

Database Description:

There are 3 tables: Vehicle_data, Mileage, and Topspeed.
Vehicle_data contains: vehicle_id , vehicle_number , smoke_emission, and temperature.
Topspeed contains: vehicle_number, capacitor , cc , and top_speed .
Mileage contains: vehicle_number, make , type  and mileage .
The vehicle_id column is only present in the Vehicle_data table.

The vehicle_id column is only present in the Vehicle_ID table.

Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

'Vehicle_data','Topspeed','Mileage'

Some examples of SQL queries that correspond to questions are:
"Find the average mileage for vehicles with a top speed greater than 200 km/h." : "SELECT Mileage.vehicle_number,Mileage.AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehicle_number = Topspeed.vehicle_number WHERE top_speed > 200 group by Mileage.vehicle_number;",
"List the vehicle model, make for vehicles with a smoke emission between 40 and 60 grams/km.": "SELECT type, make  FROM Mileage JOIN Vehicle_data ON Mileage.vehicle_number = Vehicle_data.vehicle_number WHERE Vehicle_data.smoke_emission BETWEEN 40 AND 60;",
"Retrieve the vehicle IDs and temperatures for vehicles with an engine displacement (cc) of 1500 or higher." : "SELECT vehicle_id, temperature FROM Vehicle_data JOIN Topspeed ON Vehicle_data.vehicle_number = Topspeed.vehicle_number WHERE cc >= 1500;",
"Calculate the total number of vehicles with a mileage of less than 15 km/L.":"SELECT COUNT(*) AS total_vehicles FROM Mileage WHERE mileage < 15;",
"Find the highest smoke emission value across all vehicles.":"SELECT MAX(smoke_emission) AS highest_smoke_emission FROM Vehicle_data;",
"Show the vehicle model with the lowest temperature.":"SELECT type AS vehicle_model FROM Mileage JOIN Vehicle_data ON Mileage.vehicle_number = Vehicle_data.vehicle_number WHERE temperature = (SELECT MIN(temperature) FROM Vehicle_data);"


Question: {input}"""

PROMPT = PromptTemplate(input_variables=["input"], template=template,)



In [16]:
project_id = "iron-decorator-297513"
LOCATION = "us-central1"
dataset_id = 'tg_Dataste'
table_name = ['Vehicle_data','Topspeed','Mileage']
table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(f"bigquery://{project_id}/{dataset_id}")

llm = VertexAI(
    model_name='text-bison@001',
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
    prompt_template = PROMPT
)

db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=table_name,sample_rows_in_table_info=10)

<ipython-input-16-a35bd6d00124>:18: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=table_name,sample_rows_in_table_info=10)


In [18]:
# Define few-shot examples specific to your database
few_shots = {
"Find the average mileage for vehicles with a top speed greater than 200 km/h." : "SELECT Mileage.vehicle_number,Mileage.AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehicle_number = Topspeed.vehicle_number WHERE top_speed > 200 group by Mileage.vehicle_number;",
"List the vehicle model, make for vehicles with a smoke emission between 40 and 60 grams/km.": "SELECT type, make  FROM Mileage JOIN Vehicle_data ON Mileage.vehicle_number = Vehicle_data.vehicle_number WHERE Vehicle_data.smoke_emission BETWEEN 40 AND 60;",
"Retrieve the vehicle IDs and temperatures for vehicles with an engine displacement (cc) of 1500 or higher." : "SELECT vehicle_id, temperature FROM Vehicle_data JOIN Topspeed ON Vehicle_data.vehicle_number = Topspeed.vehicle_number WHERE cc >= 1500;",
"Calculate the total number of vehicles with a mileage of less than 15 km/L.":"SELECT COUNT(*) AS total_vehicles FROM Mileage WHERE mileage < 15;",
"Find the highest smoke emission value across all vehicles.":"SELECT MAX(smoke_emission) AS highest_smoke_emission FROM Vehicle_data;",
"Show the vehicle model with the lowest temperature.":"SELECT type AS vehicle_model FROM Mileage JOIN Vehicle_data ON Mileage.vehicle_number = Vehicle_data.vehicle_number WHERE temperature = (SELECT MIN(temperature) FROM Vehicle_data);",
"Find the vehicle id and model with lowest temperature":"SELECT Mileage.type AS vehicle_model,Vehicle_data.vehicle_id FROM Mileage JOIN Vehicle_data ON Mileage.vehicle_number = Vehicle_data.vehicle_number WHERE temperature = (SELECT MIN(temperature) FROM Vehicle_data);"
}

In [46]:
FORMAT_INSTRUCTIONS = """

Database Description:

There are 3 tables: Vehicle_data, Mileage, and Topspeed.
Vehicle_data contains: vehicle_id is the Primary key , vehicle_number , smoke_emission, and temperature.
Topspeed contains: vehicle_number is the Primary key, capacitor , cc , and top_speed .
Mileage contains: vehicle_number is the Primary key, make , type  and mileage .
The vehicle_id column is only present in the Vehicle_data table.

when ever the user is asking for the vehicle id, you should do inner join with Vehicle_ID table using vehicle_number



When ever there is Agrregate function like Average , Sum Always use the Primary key of the table along with the Select statement and group by the primary key to avoid error.
When ever there is a join , make sure to use the tablename suffix when writing the select queries to avoid ambiguity error. Example : Vehicle_data.Vehilce_number, Topspped.cc etc.



Use the following format:


Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: SQL query ONLY.
"""

In [19]:

# Utility functions for Embeddings API with rate limiting
from langchain.embeddings import VertexAIEmbeddings
import langchain
from pydantic import BaseModel
import time
from typing import List

def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)


class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

In [20]:
EMBEDDING_QPM = 15
EMBEDDING_NUM_BATCH = 2
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)


In [21]:
from google.colab import drive
drive.mount('/content/drive')
vector_save_directory = '/content/drive/MyDrive/Colab Notebooks/datafiles/chroma_vector_db/SQL_RAG'

Mounted at /content/drive


In [22]:
few_shot_docs = [Document(page_content=question, metadata={"sql_query": few_shots[question]}) for question in few_shots.keys()]

In [23]:
few_shot_docs

[Document(page_content='Find the average mileage for vehicles with a top speed greater than 200 km/h.', metadata={'sql_query': 'SELECT Mileage.vehicle_number,Mileage.AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehicle_number = Topspeed.vehicle_number WHERE top_speed > 200 group by Mileage.vehicle_number;'}),
 Document(page_content='List the vehicle model, make for vehicles with a smoke emission between 40 and 60 grams/km.', metadata={'sql_query': 'SELECT type, make  FROM Mileage JOIN Vehicle_data ON Mileage.vehicle_number = Vehicle_data.vehicle_number WHERE Vehicle_data.smoke_emission BETWEEN 40 AND 60;'}),
 Document(page_content='Retrieve the vehicle IDs and temperatures for vehicles with an engine displacement (cc) of 1500 or higher.', metadata={'sql_query': 'SELECT vehicle_id, temperature FROM Vehicle_data JOIN Topspeed ON Vehicle_data.vehicle_number = Topspeed.vehicle_number WHERE cc >= 1500;'}),
 Document(page_content='Calculate the total number of vehicl

In [ ]:
for question in few_shots.keys():
  print(question)

In [28]:
few_shots['Find the average mileage for vehicles with a top speed greater than 200 km/h.']

'SELECT Mileage.vehicle_number,Mileage.AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehicle_number = Topspeed.vehicle_number WHERE top_speed > 200 group by Mileage.vehicle_number;'

In [33]:
from langchain.vectorstores import Chroma
chroma_db = Chroma.from_documents(few_shot_docs,
                                  embeddings,
                                  persist_directory=vector_save_directory)

Waiting
...

In [37]:
chroma_db.persist()
retriever  = chroma_db.as_retriever()

In [47]:
retriever_tool = create_retriever_tool(retriever, name="sql_get_similar_examples", description="Retrieves similar SQL examples.")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    format_instructions=FORMAT_INSTRUCTIONS,
    extra_tools=[retriever_tool],
    top_k=20
)

In [43]:
query_result = agent_executor.run(
    "Find the highest smoke emission value across all vehicles"
)
print("Query Result:", query_result)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Mileage, Topspeed, Vehicle_data
Thought:The Vehicle_data table contains the smoke emission value.  I should query the schema of this table to see what columns it contains.
Action: sql_db_schema
Action Input: Vehicle_data
Observation: 
CREATE TABLE `Vehicle_data` (
	`vehicle_id` INT64, 
	`vehicle_number` STRING, 
	`smoke_emission` FLOAT64, 
	`temperature` INT64
)

/*
10 rows from Vehicle_data table:
vehicle_id	vehicle_number	smoke_emission	temperature
2	DEF456	0.5	28
11	EFGH789	0.5	26
20	012JKL	0.5	32
28	456IJKL	0.5	23
1	ABC123	0.3	32
14	QRST678	0.3	30
22	678PQR	0.3	37
30	012QRST	0.3	28
8	VWX234	0.6	29
17	123ABC	0.6	21
*/
Thought:The smoke emission column is a float64.  I should query the maximum value of this column.
Action: sql_db_query
Action Input: SELECT MAX(smoke_emission) FROM Vehicle_data
Observation: [(0.9,)]
Thought:I now know the final answer
Final Answer: The highest smoke emission

In [48]:
agent_executor.run("List the Vehicle_ID which has the mileage greater than 80")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Mileage, Topspeed, Vehicle_data
Thought:I should look at the schema of the Mileage table to see what columns I can query.
Action: sql_db_schema
Action Input: Mileage
Observation: 
CREATE TABLE `Mileage` (
	`vehicle_number` STRING, 
	`make` STRING, 
	`type` STRING, 
	`mileage` INT64
)

/*
10 rows from Mileage table:
vehicle_number	make	type	mileage
DEF456	Honda	SUV	45
MNOP	Skoda	SUV	52
678PQR	Acura	SUV	49
456DEF	Land Rover	SUV	85
890ABCD	Mini	SUV	46
PQR678	Nissan	Van	59
JKL012	Chevrolet	Coupe	34
234VWX	Mitsubishi	Sedan	35
TUVW	Peugeot	Sedan	27
ABC123	Toyota	Sedan	78
*/
Thought:The Mileage table has the mileage column. The Vehicle_data table has the vehicle_id column. I need to join the two tables on vehicle_number.
Action: sql_db_schema
Action Input: Vehicle_data
Observation: 
CREATE TABLE `Vehicle_data` (
	`vehicle_id` INT64, 
	`vehicle_number` STRING, 
	`smoke_emission` FLOAT64, 
	`temperatu

'SELECT Vehicle_data.vehicle_id FROM Vehicle_data INNER JOIN Mileage ON Vehicle_data.vehicle_number = Mileage.vehicle_number WHERE Mileage.mileage > 80'

In [49]:
query_result = agent_executor.run(
    "Find the Vehicle_ID with minimum capacitor"
)

print("Query Result:", query_result)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Mileage, Topspeed, Vehicle_data
Thought:I should query the schema of the Topspeed table because it contains the capacitor column.
Action: sql_db_schema
Action Input: Topspeed
Observation: 
CREATE TABLE `Topspeed` (
	`vehicle_number` STRING, 
	`capacitor` INT64, 
	`cc` FLOAT64, 
	`top_speed` INT64
)

/*
10 rows from Topspeed table:
vehicle_number	capacitor	cc	top_speed
YZ123	1000	1.0	100
123A	1300	1.0	100
567YZ1	1400	1.1	105
123ABC	1500	1.3	110
MNOP	1600	1.4	110
MNOP345	1800	1.4	115
JKL012	1500	1.2	120
345MNO	1700	1.5	120
123EFGH	2000	1.6	125
789C	2200	1.7	125
*/
Thought:I should query the Topspeed table to get the minimum capacitor value.
Action: sql_db_query
Action Input: SELECT vehicle_number FROM Topspeed ORDER BY capacitor LIMIT 1
Observation: [('YZ123',)]
Thought:I now know the final answer
Final Answer: SELECT vehicle_id FROM Vehicle_data WHERE vehicle_number IN (SELECT vehicle_number F